In [1]:
import csv
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

In [2]:
XPATH_HTML = './/html[@class="s1-chat-initilized Chrome 129 desktop Windows landscape"]'
XPATH_BUTTON_SEE_MORE = './/button[@class="vtex-button bw1 ba fw5 v-mid relative pa0 lh-solid br2 min-h-small t-action--small bg-action-primary b--action-primary c-on-action-primary hover-bg-action-primary hover-b--action-primary hover-c-on-action-primary pointer "]'
XPATH_CARD = './/div[@id="gallery-layout-container"]//div[@class="vtex-search-result-3-x-galleryItem vtex-search-result-3-x-galleryItem--normal vtex-search-result-3-x-galleryItem--grid pa4"]//section[@class="vtex-product-summary-2-x-container vtex-product-summary-2-x-container--shelf vtex-product-summary-2-x-container--shelf-search vtex-product-summary-2-x-containerNormal vtex-product-summary-2-x-containerNormal--shelf vtex-product-summary-2-x-containerNormal--shelf-search overflow-hidden br3 h-100 w-100 flex flex-column justify-between center tc"]//a[@class="vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--shelf vtex-product-summary-2-x-clearLink--shelf-search h-100 flex flex-column"]'
XPATH_CARD_ID = './/span[@class="vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-productBrand--shelf vtex-product-summary-2-x-brandName vtex-product-summary-2-x-brandName--shelf t-body"]'
XPATH_PAGE_PRODUCT  = './/body[@class="bg-base"]'
XPATH_CATEGORY = './/span[@class="vtex-breadcrumb-1-x-arrow vtex-breadcrumb-1-x-arrow--product-breadcrumb vtex-breadcrumb-1-x-arrow--3 vtex-breadcrumb-1-x-arrow--product-breadcrumb--3 ph2 c-muted-2"]//a[@class="vtex-breadcrumb-1-x-link vtex-breadcrumb-1-x-link--product-breadcrumb vtex-breadcrumb-1-x-link--3 vtex-breadcrumb-1-x-link--product-breadcrumb--3 dib pv1 link ph2 c-muted-2 hover-c-link"]'
XPATH_BRAND_NAME = './/div[@class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex"]//a[@class="baproar-components-0-x-brandName"]'
XPATH_PRODUCT_NAME = './/h1[@class="vtex-store-components-3-x-productNameContainer vtex-store-components-3-x-productNameContainer--product-name mv0 t-heading-4"]//span[@class="vtex-store-components-3-x-productBrand vtex-store-components-3-x-productBrand--product-name "]'
XPATH_PRODUCT_RATE_SELLER = './/a[@class="baproar-components-0-x-landingSellerLink"]//span[@class="baproar-components-0-x-sellerScorePDPRate"]'
XPATH_SELLER = './/div[@class="baproar-components-0-x-sellerScorePDPSold"]//a[@class="baproar-components-0-x-sellerScorePDPLink"]'
XPATH_SAVING_PERCENTAGE = './/div[@class="pr0 items-stretch    flex"]//span[@class="vtex-product-price-1-x-savings vtex-product-price-1-x-savings--product-price-savings"]//span[@class="vtex-product-price-1-x-savingsPercentage vtex-product-price-1-x-savingsPercentage--product-price-savings"]'
XPATH_PRICE = './/span[@class="vtex-product-price-1-x-currencyContainer vtex-product-price-1-x-currencyContainer--product-selling-price"]'
XPATH_STOCK = './/div[@class="vtex-product-availability-0-x-container flex pv2"]'
XPATH_FOOTER = './/div[@class="vtex-store-footer-2-x-footerLayout"]'


In [3]:
url_base = 'https://www.provinciacompras.com.ar/hogar?page='

In [4]:
driver = webdriver.Chrome()

In [5]:
headers = ['url', 'category', 'brand_name', 'product_name', 'seller', 'rate_seller','saving_percentage', 'price', 'stock']
data = {variable: [] for variable in headers}

In [6]:
products_not_scraped = set()

In [7]:
# Función que valida y recupera los datos
def get_element(element, xpath):
    try:
        # Buscar el elemento dentro del elemento padre
        child_element = element.find_element(By.XPATH, xpath)
        # Extraer el texto del elemento hijo
        element_text = child_element.text
        if element_text:
            return element_text
        else:
            return 'Null'
    except:
        return 'Null'

In [ ]:
for page_num in range(1, 51):
	try:
		url = f"{url_base}{page_num}"
		driver.get(url)
		driver.maximize_window()
		# Esperar a que la página cargue por completo
		WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_HTML)))

		# Recolectar todas las cards de productos
		cards = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_CARD)))

		# Recorrer cada card visible
		for i in range(len(cards)):
			try:
				# Clic en la card actual
				cards[i].click()

				# Esperar a que el body de la página esté presente
				WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, XPATH_FOOTER)))

				# Capturamos el body de la página del cual vamos a capturar los datos
				body = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="render-container render-route-store-product"]')))
				
				###########CAPTURA DE LOS DATOS###############
				url = driver.current_url
				category = get_element(body, XPATH_CATEGORY)
				brand_name = get_element(body, XPATH_BRAND_NAME)
				product_name = get_element(body, XPATH_PRODUCT_NAME)
				seller = get_element(body, XPATH_SELLER)
				rate_seller = get_element(body, XPATH_PRODUCT_RATE_SELLER)
				saving_percentage = get_element(body, XPATH_SAVING_PERCENTAGE)
				price = get_element(body, XPATH_PRICE)
				stock = get_element(body, XPATH_STOCK)
				###########CAPTURA DE LOS DATOS###############

				data['url'].append(url)
				data['category'].append(category)
				data['brand_name'].append(brand_name)
				data['product_name'].append(product_name)
				data['seller'].append(seller)
				data['rate_seller'].append(rate_seller)
				data['saving_percentage'].append(saving_percentage)
				data['price'].append(price)
				data['stock'].append(stock)

				# Volver a la página anterior
				driver.back()
				driver.implicitly_wait(1)
				# Reencontrar las cards después de volver a la página anterior (DOM se actualiza)
				cards = WebDriverWait(driver, 5).until(EC.
				presence_of_all_elements_located((By.XPATH, XPATH_CARD)))
			except(TimeoutException, NoSuchElementException, StaleElementReferenceException) as error:
				products_not_scraped.append({'product_name': product_name, 'url': url, 'page_num': page_num})
				continue
	except (TimeoutException, NoSuchElementException, StaleElementReferenceException) as error:
		print(f'Error al cargar la página {page_num}')
		continue

In [11]:
df = pd.DataFrame(data)

In [12]:
df.head()

,url,category,brand_name,product_name,seller,rate_seller,saving_percentage,price,stock
0,https://www.provinciacompras.com.ar/colchon-so...,Sommiers,Simmons,Colchón Sommier Simmons Backcare Hotel Bilt,Simmons,Null,67%,$ 636.502,Stock: +50
1,https://www.provinciacompras.com.ar/colchon-si...,Colchones,Simmons,Colchón Simmons Backcare Hotel Bilt,Simmons\n2.5,2.5,67%,$ 444.357,Stock: +50
2,https://www.provinciacompras.com.ar/sommier-y-...,Sommiers,La Espumeria,Sommier y colchon Indie de Resortes,La Espumeria,Null,57%,$ 477.850,Stock: +50
3,https://www.provinciacompras.com.ar/sommier-y-...,Colchones,Sealy,Sommier Y Colchon Sealy Linen,SommierCenter,Null,41%,$ 565.699,Stock:\n40
4,https://www.provinciacompras.com.ar/hidrolavad...,Herramientas de jardín,Lusqtoff,Hidrolavadora Lusqtoff 105bar 1200w Alta Presi...,Style Store,4,45%,"$ 153.999,45",Stock: +50


In [11]:
# for column in df.select_dtypes(include=['object']).columns:
#     df[column] = df[column].astype(str).str.replace('\n', '')

In [15]:
# df.to_csv('../outputs/output_.csv', index=False)

In [13]:
# df.to_json('./outputs/output_.json')